<a href="https://colab.research.google.com/github/mjdileep/Model-Explorer/blob/main/Qwen_0_5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -q
!pip install -U transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

device = "cuda" # the device to load the model onto
prompt = """Write an essay on the possibility of the existence of Boron-based life forms and what are the odds?"""
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-0.5B-Chat",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
%%time
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
t = time.time()
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=4000,
    temperature=0.7,
    do_sample=True
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
tps  = len(generated_ids[0])/(time.time()-t)
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


CPU times: user 11.4 s, sys: 21.3 ms, total: 11.4 s
Wall time: 11.4 s


In [9]:
print(f"Peak memory usage: {torch.cuda.max_memory_allocated()*1e-9:.2f} Gb, Tokens per Second(TPS): {tps:.2f}\n")

Peak memory usage: 1.26 Gb, Tokens per Second(TPS): 31.22



In [10]:
print(response)

Boron is a naturally occurring element that is abundant in Earth's atmosphere and oceans, making it an ideal candidate for studying its effects on life. Boron-based life forms have been proposed as potential candidates for future research due to their unique properties and potential applications in various fields such as medicine, energy production, and environmental management.

One of the most promising possibilities of boron-based life forms is the development of new types of organisms with enhanced resistance to extreme conditions. Boron has been found to be highly resistant to both radiation and heat, which makes it an ideal material for creating biological structures that can withstand these conditions. For example, boron-based materials could be used to create highly durable and lightweight structures that can withstand exposure to extreme temperatures and pressures.

Another possibility is the development of new forms of life that are more efficient at photosynthesis. Boron has